# 김프를 구하기 위해 코딩에 있어서 발생하는 간격 문제.
업비트 비트코인 시세 갱신이(즉, 체결)이 바이낸스에 비해 느립니다. 따라서 정확한 김프계산에 있어 문제가 발생합니다.(김프가 실시간으로 최신화 되어야 하는데 그렇지 못함.)<br>
kimp_client 코루틴에서 업비트의 데이터가 들어오지 않으면 바이낸스의 데이터를 기다리는 binance_queue.get()도 실행되지 않기 때문에 딜레이가 발생한다는 것입니다.<br>
`즉, 위의 말은 await때문에 큐가 비어있으면(upbit 데이서 갱신이 느려서 큐가 빔) 다음 줄로 넘어가지 못함.`

```
async def kimp_client(upbit_queue, binance_queue):
    while True:
        upbit_data = await upbit_queue.get()
        binance_data = await binance_queue.get() 
        binance_krw = binance_data * 1330
        kimp = round(((upbit_data - binance_krw) / binance_krw) * 100, 2)
        print(f"Kimp: {kimp} %")
```

# 위 문제를 해결하기 위해 몇가지 방안이 있습니다.

**첫번쨰.<br>**
비동기 타임아웃 설정: asyncio.wait_for() 함수를 사용하여 일정 시간 내에 데이터가 들어오지 않으면 다음 작업을 진행하도록 할 수 있습니다.<br>
```
async def kimp_client(upbit_queue, binance_queue):
    while True:
        try:
            upbit_data = await asyncio.wait_for(upbit_queue.get(), timeout=0.1)
            binance_data = await binance_queue.get()
        except asyncio.TimeoutError:
            continue  # or use the last known upbit_data
        
        binance_krw = binance_data * 1330
        kimp = round(((upbit_data - binance_krw) / binance_krw) * 100, 2)
        print(f"Kimp: {kimp} %")
```

**두번쨰.<br>**
마지막으로 알려진 데이터 사용: 업비트 데이터가 늦게 들어오더라도 바이낸스 데이터를 계속 처리하고 마지막으로 알려진 업비트 데이터를 사용할 수 있습니다.<br>
```
last_upbit_data = None
last_binance_data = None

async def kimp_client(upbit_queue, binance_queue):
    global last_upbit_data, last_binance_data
    
    while True:
        if not upbit_queue.empty():
            last_upbit_data = await upbit_queue.get()
            
        if not binance_queue.empty():
            last_binance_data = await binance_queue.get()
        
        if last_upbit_data is not None and last_binance_data is not None:
            binance_krw = last_binance_data * 1330
            kimp = round(((last_upbit_data - binance_krw) / binance_krw) * 100, 2)
            print(f"Kimp: {kimp} %")

        await asyncio.sleep(0.1)  # Sleep to prevent high CPU usage
```

여기서 마지막으로 알려진 데이터 사용을 써야 훨씬 좋음.
